# National Weather Service RADAR Maps

### Justin Richling  4/18/2017

#### https://radar.weather.gov/ridge/Conus/index.php

In [ ]:
import os
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import time,urllib,urllib2,cStringIO,logging,datetime,webbrowser,\
IPython.display,shutil
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
from scipy.io import netcdf
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup
from IPython.core.display import Image
from PIL import Image as PILImage
import sys
from matplotlib.pyplot import *

In [ ]:
from datetime import datetime
print "Current Hour: "+str(datetime.utcnow().hour)+"Z UTC"

In [3]:
RegionNames = ["Pacific Northwest","Northern Rockies","Upper Miss. Valley","Central Great Lakes",\
    "Northeast","Southeast","Southern Miss. Valley","Southern Plains","Southern Rockies",\
    "Pacific Southwest","Hawaii","Alaska","Cont. US"]

# ExtNames are used by NWS's webpage 
ExtNames = ["pacnorthwest","northrockies","uppermissvly","centgrtlakes","northeast","southeast",\
    "southmissvly","southplains","southrockies","pacsouthwest","hawaii","alaska","latest"]

# This list will be populated by all the full webpage addresses
RegionPageName = []
for j in range(len(RegionNames)):
    RegionPageName.append("http://radar.weather.gov/ridge/Conus/RadarImg/"+ExtNames[j]+".gif")

In [15]:
RegionPageName

['http://radar.weather.gov/ridge/Conus/RadarImg/pacnorthwest.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/northrockies.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/uppermissvly.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/centgrtlakes.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/northeast.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/southeast.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/southmissvly.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/southplains.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/southrockies.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/pacsouthwest.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/hawaii.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/alaska.gif',
 'http://radar.weather.gov/ridge/Conus/RadarImg/latest.gif']

In [26]:
def NWS_Radar_Pages(page,Extname,Save_Folder,Region):
    now = datetime.now()
    print now
    Year = now.year
    Month = now.month
    Day = now.day
    # utcnow will give us 24 hr clock    
    Hour = datetime.utcnow().hour
    Minute = datetime.utcnow().minute
    # We need the month and day number to have a 0 in front for the web address
    # This will just concoctinate?? add a zero if its less than 10 
    if Month < 10:
        Month = "0"+str(Month)
    if Day < 10:
        Day = "0"+str(Day)
    if Hour < 10:
        Hour = "0"+str(Hour)
    if Minute < 10:
        Minute = "0"+str(Minute)

    try:
        print page

# Open a browser to the page
    #webbrowser.open(picture_page) 

# open the web page picture and read it into a variable
        opener1 = urllib2.build_opener()
        page1 = opener1.open(page)
        my_picture = page1.read()
        
# open file for binary write and save picture
# picture_page[-4:] extracts extension eg. .gif
# (most image file extensions have three letters, otherwise modify)
        Filename = str(Year)+"_"+str(Month)+"_"+str(Day)+\
    "_"+str(Hour)+"_"+str(Minute)+"Z"+"_"+str(Extname)+".png" 

        fout = open(Filename, "wb")
        fout.write(my_picture)
        fout.close()
    
# test to see if it saved correctly
        #webbrowser.open(Filename)                   
    except urllib2.HTTPError, err:
        if err.code == 404:
            logging.exception("No Data from website")
            pass
        else:
            raise

    file = cStringIO.StringIO(urllib.urlopen(page).read())
    img = PILImage.open(file)
    fig, ax = subplots(figsize=(15, 8))
    ax.set_title(str(Region)+" Radar: "+str(Year)+" "+\
                 str(Month)+" "+str(Day)+" "+str(Hour)+str(Minute)+"Z")
    ax.imshow(img, interpolation='nearest')
    plt.axis('off')
    tight_layout()
    #plt.show()
    plt.close()
    
    return Filename

In [ ]:
Inputz = raw_input("How many regions would you like to track? (A)ll to run all regions ") 
MyInput = []
if Inputz == "A":
    MyInput = RegionNames
    #MyInput = ListRegion  
    print MyInput
else:
    Inputz = int(Inputz)
    for i in range(Inputz):
    # Give a list of region names to be used as input by user
        for h in range(len(RegionNames)):
            #print "(for reference on NPC's website)"+"\n\n"+"Map sector number: "+str(butt[h])
            print str(RegionNames[h])
        print
        MyInput.append(raw_input("Which Region: "))
        print "\n"+str(MyInput)+"\n"
    
Time = input("How many hours for the loop? ")

now = datetime.now()
Month = now.month
Year = now.year
Day = now.day
    # utcnow will give us 24 hr clock    
Hour = datetime.utcnow().hour
Minute = datetime.utcnow().minute
    # We need the month and day number to have a 0 in front for the web address
    # This will just concoctinate?? add a zero if its less than 10     
if Month < 10:
    Month = "0"+str(Month)
if Day < 10:
    Day = "0"+str(Day)
    
Name = raw_input("Where would you like to save the RADAR folders? ")

# The 3 in the range argument needs to be there to reflect the fact that how ever many
# hours the user wants, we need to multiply it by an amount that when multiplied to 
# the frequency (refresh rate: 20 min) to get the main for-loop to execute properly since
# it doesn't know that we are taking about time; base 60, not 10

# If Time = 2 hours, that's 120 minutes. At refresh rate of 20 mins (in time argument below),
# we need the range to essentiall be 6. So Time (2) x 3 = 6. 
# Check. 
for g in range(3*Time):
    #print g
    for i in range(len(MyInput)):    
    
# Check to see if Input (user's input) is in our list of NOAA regions
        if MyInput[i] in RegionNames:
    # If Input is in our list, then set j equal to that numbered index
    # j will be an int!!
            j = RegionNames.index(MyInput[i])
            print "\n"+"Index Number: "+str(j)
    # We will use j in a moment, it is the list entry number of input region
    
    # We want to store this particular storm in it's designated folder    
        Name_new = str(Name)+str(Year)+"_"+str(Month)+"_"+str(Day)+"_"+str(MyInput[i])
        print Name_new
        
# Checking to see if the folder exists already (os.path.isdir) and if not, create one (os.makedirs)
# Then redirect (os.chdir) to that folder so later the created image can be automatically saved to it
        if not os.path.isdir(Name_new):
            os.makedirs(Name_new)
        os.chdir(Name_new)
    
        NWS_Radar_Pages(RegionPageName[j],MyInput[i],Name,Name_new)
    # come back and capture an updated (hopefully) RADAR map every 20 minutes
    time.sleep(20*60)

# The time.sleep() at the end of the loop will allow for the time desired between plots
print "All done."